<a href="https://colab.research.google.com/github/PratikshaKage/Voice-Cloning/blob/main/voice_cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

import os
from google.colab import files

In [ ]:
CUSTOM_VOICE_NAME="custom"
custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:

from torchaudio import transforms
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
custom_audio_file = "audio2.wav"
custom_audio, = load_audio(custom_audio_file, sampling_rate=44100)

# Extract the features of the custom audio file
melspectrogram = transforms.MelSpectrogram(sample_rate=44100, n_mels=128)
features = melspectrogram(custom_audio)

# Create a tensor that represents the features of the custom audio file
tensor = features.unsqueeze(0)
text = "Machine learning is a transformative field of artificial intelligence that empowers computers to learn and improve from experience without being explicitly programmed."

preset = "fast"

conditioning_latents = tensor.clone()
conditioning_latents[0, 0] *= 6  # Increase the pitch
conditioning_latents[0, 1] *= 0.50  # Decrease the voicing
conditioning_latents[0, 2] *= 6   # Increase the rounding of vowels
conditioning_latents[0, 3] *= 0.75  # Decrease the aspiration of consonants
conditioning_latents[0, 10] = 0.50 # Decrease speech rate
conditioning_latents[0, 11] = 2 # Increase intonation
conditioning_latents[0, 12] = 3  # Increase syllabic rate
conditioning_latents[0, 13] = 6 # Increase word stress
conditioning_latents[0, 15] = 1 # Increase breath intake
conditioning_latents[0, 16] = 20.5# Increase accent


gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)

torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)

IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')